In [1]:
from datetime import datetime
from uuid import uuid4


now = datetime.now()
username = "ponakilan"
with open(f"{username}_{now}_{uuid4()}", "w") as f:
    pass

In [2]:
import os

files = os.listdir("Files")

files_d = {
    "username": [],
    "time": [],
    "job_id": []
}
for file in files:
    username, time, uuid = file.split('_')
    files_d['username'].append(username)
    files_d['time'].append(time)
    files_d['job_id'].append(uuid)
files_d

ValueError: not enough values to unpack (expected 3, got 1)

In [4]:
import pandas as pd

df = pd.DataFrame(files_d)
df

,username,time,job_id


In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Jobs-Scheduler').getOrCreate()

input_dir = "Files/Input"
output_dir = "Files/Output"
username = "ponakilan"

files_inp = {
    "username": [],
    "job_id": [],
    "in_time": []
}
for file in os.listdir(input_dir):
    username, time, uuid = file.split('_')
    files_inp['username'].append(username)
    files_inp['in_time'].append(time)
    files_inp['job_id'].append(uuid)

files_out = {
    "username": [],
    "job_id": [],
    "out_time": []
}
for file in os.listdir(output_dir):
    username, time, uuid = file.split('_')
    files_out['username'].append(username)
    files_out['out_time'].append(time)
    files_out['job_id'].append(uuid)

inp_df = spark.createDataFrame(pd.DataFrame(files_inp))
out_df = spark.createDataFrame(pd.DataFrame(files_out))

df = inp_df.join(out_df.drop("username"), "job_id", 'left')
df

DataFrame[job_id: string, username: string, in_time: string, out_time: string]

In [ ]:
import json

results = json.loads(df.toJSON().first())

for key in results:
    print(results[key])

# To decode the entire DataFrame iterate over the result
# of toJSON()

def print_rows(row):
    data = json.loads(row)
    for key in data:
        print("{key}:{value}".format(key=key, value=data[key]))

In [14]:
import json

jobs = df.toPandas().to_json(orient="records")
jobs = json.loads(jobs)
jobs

[{'job_id': 'ce1c7b28-5741-4f6b-906e-91e1516991e2',
  'username': 'ponakilan',
  'in_time': '2024-12-23 06:54:26.920982',
  'out_time': '2024-12-23 06:54:26.920982'},
 {'job_id': '50769605-2cd5-4e9d-9fd8-666b76d9827e',
  'username': 'ponakilan',
  'in_time': '2024-12-23 06:54:27.195678',
  'out_time': None},
 {'job_id': '02d401b3-255f-4dea-905e-1babaf7bdf41',
  'username': 'ponakilan',
  'in_time': '2024-12-23 06:54:26.195974',
  'out_time': '2024-12-23 06:54:26.195974'},
 {'job_id': 'f78c1148-f885-4bf9-a6d0-8b299b649dc1',
  'username': 'ponakilan',
  'in_time': '2024-12-23 06:54:27.559175',
  'out_time': None}]

/tmp/ipykernel_1916/2161352320.py:1: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  pd.read_json(f"{str(jobs).replace('[', '{').replace(']', '}')}")


ValueError: Expected object or value